# Integration

**[Sanorama Hie et al., 2019](https://www.nature.com/articles/s41587-019-0113-3)**  
[GitHub](https://github.com/brianhie/scanorama)  
[Tutorial external API](https://scanpy-tutorials.readthedocs.io/en/latest/spatial/integration-scanorama.html)  
[External external API tutorial](https://scanpy.readthedocs.io/en/stable/generated/scanpy.external.pp.scanorama_integrate.html)

A [fix](https://github.com/theislab/single-cell-tutorial/issues/60) to run [scran pooling normalization](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-016-0947-7) computeSumFactors in current python environment. 

In [ ]:
import scanpy as sc
import scanorama

import numpy as np
import pandas as pd

import os

In [ ]:
# Working directory 
os.chdir('/research/peer/fdeckert/FD20200109SPLENO')

In [ ]:
# rpy2 
os.environ['R_HOME'] = '/home/fdeckert/bin/miniconda3/envs/p.3.8.12-FD20200109SPLENO/lib/R'

# Communication 
import rpy2.rinterface_lib.callbacks
import logging
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR) # Ignore R warnings

# Convertion 
from rpy2.robjects import pandas2ri
import anndata2ri

# Activate conversion globaly 
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
%%R
library(scran)

In [ ]:
# Plotting 
import rpy2.robjects as robjects
color_load = robjects.r.source('plotting_global.R')
color = dict()
for i in range(len(color_load[0])):
    color[color_load[0].names[i]] = {key : color_load[0][i].rx2(key)[0] for key in color_load[0][i].names}

sc.set_figure_params(figsize=(5, 5))

# Import AnnData

In [ ]:
adata = sc.read_h5ad('data/object/qc.h5ad')
adata = adata[adata.obs['doublet_class']=='Singlet'].copy()

In [ ]:
def set_color(categories): 
    
    categories = [x for x in categories if x in list(adata.obs.columns)]

    for category in categories: 
        
        adata.obs[category] = pd.Series(adata.obs[category], dtype='category')
        
        keys = list(color[category].keys())
        keys = [x for x in keys if x in list(adata.obs[category])]

        adata.obs[category] = adata.obs[category].cat.reorder_categories(keys)
        adata.uns[category+'_colors'] = np.array([color[category].get(key) for key in keys], dtype=object)
        
# Set colors
set_color(list(color.keys()))

# Subset adata by sample_group

In [ ]:
adata_sub = dict()
for sample_group in adata.obs['treatment'].unique():
    adata_sub[sample_group] = adata[adata.obs['treatment']==sample_group].copy()
adata_sub = list(adata_sub.values())

# Compute scran size factor

In [ ]:
def scran_input(adata):
    
    adata_pp = adata.copy()
    
    # For computeSumFactors we need a good coverage for each gene
    sc.pp.filter_genes(adata_pp, min_cells=20) 
    
    # Compute groups 
    sc.pp.normalize_per_cell(adata_pp, counts_per_cell_after=1e6)
    sc.pp.log1p(adata_pp)
    sc.pp.pca(adata_pp, n_comps=15, svd_solver='arpack')
    sc.pp.neighbors(adata_pp)
    sc.tl.louvain(adata_pp, key_added='groups', resolution=1)
    
    input_groups = adata_pp.obs['groups']
    
    # Set count matrix
    data_mat = adata.X.T
    
    data = {'input_groups': input_groups, 'data_mat': data_mat}
    
    return(data)

In [ ]:
scran_input = [scran_input(x) for x in adata_sub]

### NaCl

In [ ]:
data_mat = scran_input[0]['data_mat']
input_groups = list(scran_input[0]['input_groups'])

In [ ]:
%%R -i data_mat -i input_groups -o size_factors
size_factors = sizeFactors(computeSumFactors(SingleCellExperiment(list(counts=data_mat)), clusters=input_groups, min.mean=0.1))

In [ ]:
adata_sub[0].obs['size_factors'] = size_factors

### CpG

In [ ]:
data_mat = scran_input[1]['data_mat']
input_groups = list(scran_input[1]['input_groups'])

In [ ]:
%%R -i data_mat -i input_groups -o size_factors
size_factors = sizeFactors(computeSumFactors(SingleCellExperiment(list(counts=data_mat)), clusters=input_groups, min.mean=0.1))

In [ ]:
adata_sub[1].obs['size_factors'] = size_factors

# Normalize 

In [ ]:
adata_sub[0].X /= adata_sub[0].obs['size_factors'].values[:,None]
sc.pp.log1p(adata_sub[0])

In [ ]:
adata_sub[1].X /= adata_sub[1].obs['size_factors'].values[:,None]
sc.pp.log1p(adata_sub[1])

# Scale data 

In [ ]:
sc.pp.scale(adata_sub[0])
sc.pp.scale(adata_sub[1])

# Scanorama

In [ ]:
# Run Scanorama
scanorama.integrate_scanpy(adata_sub, dimred=200, knn=50, verbose=True)

# Concatenate scanorama output 
X_scanorama = [ad.obsm['X_scanorama'] for ad in adata_sub]
X_scanorama = np.concatenate(X_scanorama)

obs_names = [ad.obs_names for ad in adata_sub]
obs_names = np.concatenate(obs_names)
all(obs_names==adata.obs_names)

# Add X_scanorama integration to adata 
adata.obsm["X_scanorama"] = X_scanorama

In [ ]:
# # Dimensional reduction and clustering 
sc.pp.neighbors(adata, n_neighbors=100, use_rep='X_scanorama')
sc.tl.leiden(adata, resolution=1)
sc.tl.louvain(adata, resolution=1)
sc.tl.umap(adata)

# Plot 
sc.pl.umap(adata, color=['louvain', 'leiden', 'tissue', 'treatment', 'label_fine_haemosphere', 'sample_rep', 'cc_phase_class', 'pHb_RNA', 'pRb_RNA'], wspace=0.5, ncols=3)

# Scanorama (8000 HVG)

In [ ]:
adata_sct = sc.read_h5ad('data/object/so_sct.h5ad')

In [ ]:
hvg_8000 = list(adata_sct.uns['hvg_int_8000'])

In [ ]:
adata_sub[0] = adata_sub[0][:, hvg_8000]
adata_sub[1] = adata_sub[1][:, hvg_8000]

In [ ]:
# Run Scanorama
scanorama.integrate_scanpy(adata_sub, dimred=200, knn=50, verbose=True)

# Concatenate scanorama output 
X_scanorama = [ad.obsm['X_scanorama'] for ad in adata_sub]
X_scanorama = np.concatenate(X_scanorama)

obs_names = [ad.obs_names for ad in adata_sub]
obs_names = np.concatenate(obs_names)
all(obs_names==adata.obs_names)

# Add X_scanorama integration to adata 
adata.obsm["X_scanorama"] = X_scanorama

In [ ]:
# Dimensional reduction and clustering 
sc.pp.neighbors(adata, n_neighbors=100, use_rep='X_scanorama')
sc.tl.leiden(adata, resolution=1)
sc.tl.louvain(adata, resolution=1)
sc.tl.umap(adata)

# Plot 
sc.pl.umap(adata, color=['louvain', 'leiden', 'tissue', 'treatment', 'label_fine_haemosphere', 'sample_rep', 'cc_phase_class', 'pHb_RNA', 'pRb_RNA'], wspace=0.5, ncols=3)

# Scanorama (4000 HVG)

In [ ]:
adata_sct = sc.read_h5ad('data/object/so_sct.h5ad')

In [ ]:
hvg_4000 = list(adata_sct.uns['hvg_int_4000'])

In [ ]:
adata_sub[0] = adata_sub[0][:, hvg_4000]
adata_sub[1] = adata_sub[1][:, hvg_4000]

In [ ]:
# Run Scanorama
scanorama.integrate_scanpy(adata_sub, dimred=200, knn=50, verbose=True)

# Concatenate scanorama output 
X_scanorama = [ad.obsm['X_scanorama'] for ad in adata_sub]
X_scanorama = np.concatenate(X_scanorama)

obs_names = [ad.obs_names for ad in adata_sub]
obs_names = np.concatenate(obs_names)
all(obs_names==adata.obs_names)

# Add X_scanorama integration to adata 
adata.obsm["X_scanorama"] = X_scanorama

In [ ]:
# Dimensional reduction and clustering 
sc.pp.neighbors(adata, n_neighbors=100, use_rep='X_scanorama')
sc.tl.leiden(adata, resolution=1)
sc.tl.louvain(adata, resolution=1)
sc.tl.umap(adata)

# Plot 
sc.pl.umap(adata, color=['louvain', 'leiden', 'tissue', 'treatment', 'label_fine_haemosphere', 'sample_rep', 'cc_phase_class', 'pHb_RNA', 'pRb_RNA'], wspace=0.5, ncols=3)

# Scanorama (2000 HVG)

In [ ]:
adata_sct = sc.read_h5ad('data/object/so_sct.h5ad')

In [ ]:
hvg_2000 = list(adata_sct.uns['hvg_int_2000'])

In [ ]:
adata_sub[0] = adata_sub[0][:, hvg_2000]
adata_sub[1] = adata_sub[1][:, hvg_2000]

In [ ]:
# Run Scanorama
scanorama.integrate_scanpy(adata_sub, dimred=200, knn=50, verbose=True)

# Concatenate scanorama output 
X_scanorama = [ad.obsm['X_scanorama'] for ad in adata_sub]
X_scanorama = np.concatenate(X_scanorama)

obs_names = [ad.obs_names for ad in adata_sub]
obs_names = np.concatenate(obs_names)
all(obs_names==adata.obs_names)

# Add X_scanorama integration to adata 
adata.obsm["X_scanorama"] = X_scanorama

In [ ]:
# Dimensional reduction and clustering 
sc.pp.neighbors(adata, n_neighbors=100, use_rep='X_scanorama')
sc.tl.leiden(adata, resolution=1)
sc.tl.louvain(adata, resolution=1)
sc.tl.umap(adata)

# Plot 
sc.pl.umap(adata, color=['louvain', 'leiden', 'tissue', 'treatment', 'label_fine_haemosphere', 'sample_rep', 'cc_phase_class', 'pHb_RNA', 'pRb_RNA'], wspace=0.5, ncols=3)